In [1]:
import numpy as np
import networkx as nx
import sympy
import matplotlib.pyplot as plt
from sympy.utilities.iterables import multiset_permutations
import cirq
import scipy
import scipy.optimize as optimize
import timeit
import os
import pandas as pd
import itertools
from cirq.circuits import InsertStrategy

#defining Pauli-Matrices
X = np.array([[0,1],[1,0]])
Y = np.array([[0,-1j],[1j,0]])
Z = np.array([[1,0],[0,-1]])
I = np.array([[1,0],[0,1]])

# interactions class

Consist of functions that perform on the interactions.

INPUT:


    -N : amount of qubits
    -interactions: A list of lists that contain interactions and corresponding coupling strengths.
          Example: [['P1P2...PN',J1], ['P1P2...PN',J2],....], where P is one of {X,Y,Z,I}.
          One can leave identities out of the expression. J is the strength of the interaction and thus a number.
          
## Functions:

### .split()

Splits the interactions as given in the input into two lists: Paulis, strings giving the Pauli operations. And positions, a list of the same length giving the qubit number on which the pauli operation acts.

OUTPUT:

    -returns 2 lists of same length, on containing operations, the other containing the corresponding positions.

### Interactions_to_H() 

Is a function for construncting the numpy array hamiltonian from the interactions.

OUTPUT:

    -H: Hamiltonian (2^N,2^N)-d array
### exact_eigen

Calculates the eigenvalue and eigenvector of the Hamiltonian exactly

OUTPUT:

    - ground_energy: Lowest eigenvalue of Hamiltonian
    -ground_state: Eigenstate corresponding to lowest eigenvalue

In [19]:
class interactions:
    def __init__(self, N , interactions):
        self.N = N
        self.interactions = interactions
        
        
    def split(self):
        paulis = []
        positions = []
        for i in range(len(self.interactions)):
            pauli_lst = []
            pos_lst = []

        
            #creating lists of operators and corresponding positions
            prev_int = False
            for k in self.interactions[i][0]:
                if k.isdigit():
                    if not prev_int:
                        pos_lst.append(k)
                    else:
                        pos_lst[-1] += k
                    prev_int = True
                else:
                    pauli_lst.append(k)
                    prev_int = False
            paulis.append(pauli_lst)
            positions.append(pos_lst)
        return positions, paulis
    
    
    
    def Hamiltonian(self):
        H = np.zeros((2**self.N,2**self.N)).astype('complex128')
        positions, paulis = self.split()
        for i in range(len(self.interactions)):
            Kron = 1

            #computes tensorproduct for every interaction
            n=0
            for j in range(N):

                if j+1 == int(positions[i][n]):
                    M = eval(paulis[i][n])
                    if n+1 < len(positions[i]):
                        n+=1
                else:
                    M = I
                Kron = np.kron(Kron,M)

            H += Kron*self.interactions[i][1]
        return H
    
    def exact_eigen(self):
        H = self.Hamiltonian()
        eigen = np.linalg.eig(H)
        index = np.argmin(eigen[0])
        ground_energy = eigen[0][index]
        ground_state = eigen[1][:,index]
        return ground_state, ground_energy


# k class
class for performing operations on k vectors.

INPUT:


    -N : amount of qubits
    -interactions: A list of lists that contain interactions and corresponding coupling strengths.
          Example: [['P1P2...PN',J1], ['P1P2...PN',J2],....], where P is one of {X,Y,Z,I}.
          One can leave identities out of the expression. J is the strength of the interaction and thus a number.
    -k: a list of len(interactions), with a {k_i mod 2  = 1} on location i if that coupling is turned on. 
        {k_i mod 2 = 0} otherwise
        
## Functions in k class

### .state()

Calculates the resulting state of k applied to a $|{0}>^N$ state.

OUTPUT:

        - state: list of length N, with 1 on position if k flips the i th qubit.
            
        - a: stores the phase of the state. 1 if complex, 0 if not.
### .con()
Calculates whether the diagram corresponding to k is connected or not.
See https://arxiv.org/pdf/1907.08157.pdf for further details.

OUTPUT:

        - True: if connected
        - False: if disconnected
### .generator()
Calculates the generator belonging to k

OUTPUT:
        
        -T: (2^N,2^N) numpy array representing the generator

In [78]:
class k:
    def __init__(self, N, interactions, k):
        self.N = N
        self.interactions = interactions
        self.k = k
    
    
    def state(self):
        positions, paulis = interactions(self.N, self.interactions).split()
        starting_state = -np.ones(self.N)
        a = -1
        for i in range(len(k)):
            if k[i] %2 != 0:
                for j in range(len(positions[i])):
                    if paulis[i][j] == 'X':
                        starting_state[int(positions[i][j])-1] *= -1
                    elif paulis[i][j]=='Y':
                        starting_state[int(positions[i][j])-1] *= -1
                        a *=-1
                    
                        
        a = (a+1)/2
        state = (starting_state+1)/2
        return a, list(state)
    
    
    def con(self):
        positions = interactions(self.N, self.interactions).split()[0]
        connect_on = []
        for i in range(len(k)):
                if k[i] != 0:
                    connect_on.append(positions[i])
        v = connect_on[0]
        j = 0
        while len(connect_on)!= j:
            connect_set = set(connect_on[j])
            v_set = set(v)

            if (v_set&connect_set):
                v=v+connect_on.pop(j)
                j=0
                if connect_on == []:
                    return True
            else:
                j+=1
        return False
    
    
    def generator(self):
        state = self.state(self.k)
        T = 1
        complx = False
        string = ""
        if state[0] == 1:
            complx = True
        for j in state[1]:
            if j == 1:
                if not complx:
                    M = Y
                    string.append("Y")
                    complx = True
                else:
                    M = X
                    string.append("X")
            else:
                M = I
                string.append("I")
            T = np.kron(T,M)
        return T, string

# Building the ansatz

## Choosing generators
### k_vecs()

INPUT:

    -PT_order: integer, representing up till which order in perturbation theory one wants to consider.

OUTPUT:

    -k_vecs: list of allowed k's. Where allowed means, with a connected diagram 
    and not with an order greater or equal to a k that produces exactly the same state.
    
### generators()
Takes all k's in k_vecs(), and turns them into numpy array generators

INPUT:

        -P_n: Number of generators one wants to consider, must be smaller

OUTPUT:

        generators: list of length P_n, with (2^N,2^N) arrays representing the generators


In [ ]:
def k_vecs(N, interactions, PT_order):
    k_length = len(interactions)
    k_vecs = []
    k_vecs_allowed = []
    states = [k(np.zeros(k_length)).state()]
    max_gates = min([PT_order, 2*k_length])
    for i in range(1,max_gates+1):
        k = []
        order = i
        if order%2==1:
            k = k+[1]
            order -= 1
        p = order/2
        k +=int(p)*[2]+int(k_length-len(k)-p)*[0]
        k_vecs.append(k)
        k_1 = k.copy()
            
        while 2 in k_1 and 0 in k_1:
            k_1.remove(2)
            k_1.remove(0)
            k_1+=[1,1]
            k_add= k_1.copy()
            k_vecs.append(k_add)
    for i in k_vecs:
        perms = list(multiset_permutations(i))
        for j in perms:
            if k(j).con() and (k(j).state() not in states):
                states.append(k(j).state())
                k_vecs_allowed.append(j)
    return(k_vecs_allowed)

In [ ]:
def generators(N, interactions, PT_order, P_n):
    k = k_vecs(N, interactions, PT_order)
    P_strings = []
        
    for i in range(self.P_n):
        T= k(N, interactions, k[i]).generator()
        P_strings.append(T)
    return P_strings

In [80]:
def generator_conversion(strings):
    generators = []
    for i in strings:
        Kron = 1

        #computes tensorproduct for every interaction
        n=0
        for j in range(len(i)):
            M = eval(i[j])
            Kron = np.kron(Kron,M)
        generators.append(Kron)

    return generators
            

## The form of the ansatz; structuring the generators
In this section we define different ways of structuring the ansatz.
All of them have at least the following input:

INPUT:
        
        -P_strings: list of (2^N,2^N) numpy array generators 
        -theta: list of angles, where theta_i corresponds to the i^th generator in P_strings
        -N: number of qubits
Note that for different ansatzes, there might extra inputs required.

### QCA_ansatz()
This ansatz will arrange the generators in the following way:

$U = e^{i\theta_NP_n}...e^{i\theta_2P_2}e^{i\theta_1P_1}$
### UCC_ansatz()
This ansatz cannot be implemented on actual hardware, and is implemented theoretically by the following scheme:

$U = e^{i(\theta_1P_1 + \theta_2P_2 + .... + \theta_NP_N)}$

### UCC_Trot_con()
This is an approximation of the UCC ansatz, which basically repeats the QCA_ansatz Trot_order times:

$U = \prod_{Trot_{order}} e^{i\theta_NP_n}...e^{i\theta_2P_2}e^{i\theta_1P_1}$

In this ansatz the angles throughout the different Trotter steps are constrained. In other words, the angle for gate $P_{i}$ and for $P_{i+P_n}$ is the same.
### UCC_Trot_uncon()
This ansatz is also a approximation to the UCC ansatz, but is given more freedom as the angles are relaxed for different Trotter steps. This function also takes an additional input: N_first. The first N_first gates of P_n are repeated, with all unconstrained angles:

$U = \prod_i^{\left \lfloor{len(\theta)/N_{first}}\right \rfloor} e^{i\theta_{N_{first}+i}N_{first}}...e^{i\theta_{2+i}P_{2}}e^{i\theta_{1+i}P_{1}}$

### hybrid()
### hybrid_rev()

In [4]:

def QCA_ansatz(theta,N,P_strings):
    matrix = np.identity(2**N).astype('complex128')
    for i in range(len(P_strings)):
        O = scipy.sparse.linalg.expm(1j*theta[i]*P_strings[i])
        matrix = O.dot(matrix)
    return matrix
    
    
def UCC_ansatz(theta,N,P_strings):
    matrix = np.zeros((2**N,2**N)).astype('complex128')
    for i in range(len(P_strings)):
        matrix += theta[i]*P_strings[i]
    matrix = scipy.sparse.linalg.expm(1j*matrix)
    return matrix

def UCC_Trot_con(theta,N,P_strings,Trot_order = 2):
    a = QCA_ansatz(theta, N, P_strings)
    matrix = a
    for i in range(Trot_order-1):
        matrix = matrix.dot(a)
    return matrix

def UCC_Trot_uncon(theta,N,P_strings, N_first = None):
    matrix = QCA_ansatz(theta[:N_first],N, P_strings[:N_first])
    a = len(theta)%N_first
    for i in range(1,int(np.floor(len(theta)/N_first))):
        matrix1 = QCA_ansatz(theta[i*N_first:(i+1)*N_first],N,P_strings[:N_first])
        matrix = matrix.dot(matrix1)
    matrix2 = QCA_ansatz(theta[-a:],N, P_strings[:a])
    matrix = matrix.dot(matrix2)
    return matrix

def hybrid(theta, N, P_strings, N_first, switch):
    length = len(P_strings)
    if switch < length:
        matrix1 = QCA_ansatz(theta[:switch], N,P_strings[:switch])
        matrix2 = UCC_Trot_uncon(theta[switch:], N, P_strings[:length-switch],N_first)
        matrix = matrix2.dot(matrix1)
    else:
        matrix = QCA_ansatz(theta, N,P_strings)
    return matrix

def hybrid_rev(theta, N, P_strings, N_first, switch):
    length = len(P_strings)
    if switch < length:
        matrix1 = QCA_ansatz(theta[:switch], N,P_strings[:switch])
        matrix2 = UCC_Trot_uncon(theta[switch:], N, rotate(P_strings[:length-switch],1),N_first)
        matrix = matrix1.dot(matrix2)
    else:
        matrix = QCA_ansatz(theta, N,P_strings)
    return matrix

    
    
    
def Psi(theta, ansatz, N, P_strings):
    psi = np.zeros(2**N)
    psi[0]=1
    if ansatz[0] == 'QCA':
        U = QCA_ansatz(theta, N, P_strings)
    elif ansatz[0] == 'UCC':
        U = UCC_ansatz(theta, N, P_strings)
    elif ansatz[0] == 'UCC_Trot_con':
        U = UCC_Trot_con(theta, N, P_strings,ansatz[1])
    elif ansatz[0]=='UCC_Trot_uncon':
        U = UCC_Trot_uncon(theta, N, P_strings,ansatz[1])
    elif ansatz[0]=='Hybrid':
        U = hybrid(theta, N , P_strings, ansatz[1],ansatz[2])
    elif ansatz[0]=='Hybrid_rev':
        U = hybrid_rev(theta, N, P_strings, ansatz[1], ansatz[2])
    else:
        U = np.identity(2**N)
        print('invalid ansatz')
            
    psi = U.dot(psi)
    return(psi)
    


In [ ]:
def energy(theta,H,ansatz, N, P_strings):
    psi = Psi(theta,ansatz, N, P_strings)
    return np.real(np.transpose((np.conj(psi)).dot(H.dot(psi))))
    
def VQa(H, ansatz,N, P_strings,initial_guess = np.array([None])):
    if initial_guess.any() == None or len(initial_guess)!= len(P_strings):
        initial_guess = len(P_strings)*[0]
    result = optimize.minimize(energy, initial_guess,jac = False, args = (H,ansatz,N,P_strings))#minimizer_kwargs = {"method" :'dogleg',"args":(H,ansatz,N,P_strings)}, jac = False, niter = 1, accept_test = MyBounds())
    ground_state = (Psi(result.x, ansatz,N, P_strings))
    return ground_state, result

In [ ]:
result.fun = #found ground_energy
result.x = # resulting thetas
result.nfev = #function evaluations




Format couplings $\rightarrow$TFIM example interactions = $[[X1X2, J1], [X2X3, J2], ..., [Xn-1Xn, Jn-1]]$

P_tensorpruducts() within the inter class will transform these into a list of matrices $[P_1, P_2, ..., P_n]$
, such that $U_1 = e^{i\theta_1 \pi P_1}$ will produce the state of the lowest order diagram.

$U = U_n ...U_1 = e^{i\theta_n \pi P_n}....e^{i\theta_1 \pi P_1}$

These unitaries are made and put together with the functions: QCA_anstatz($\vec{\theta}$), UCC_ansatz($\vec{\theta}$), UCC_Trot_con($\vec{\theta}$), UCC_Trot_uncon($\vec{\theta}$), hybrid and hybrid_rev($\vec{\theta}$)

Where $\vec{\theta}$ = list([$\theta_1$, $\theta_2$, ..., $\theta_n$])

The function VQa(H, ansatz,N, P_strings,initial_guess = np.array([None])) is used to minimize the groundstate energy. The initial_guess would be an array of starting thetas, and if none are given, [0,0,0,...,0] will be used.

In [ ]:
#list of generators [P1,P2,P3,P1,P2,] for TUCC e.g.